In [1]:
!wget http://www.lawrence.edu/fast/greggj/CMSC490/shakespeare.zip
!unzip shakespeare.zip

URL transformed to HTTPS due to an HSTS policy
--2023-10-15 14:24:51--  https://www.lawrence.edu/fast/greggj/CMSC490/shakespeare.zip
Resolving www.lawrence.edu (www.lawrence.edu)... 143.44.124.14
Connecting to www.lawrence.edu (www.lawrence.edu)|143.44.124.14|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www7.lawrence.edu/fast/greggj/CMSC490/shakespeare.zip [following]
--2023-10-15 14:24:51--  https://www7.lawrence.edu/fast/greggj/CMSC490/shakespeare.zip
Resolving www7.lawrence.edu (www7.lawrence.edu)... 143.44.124.14
Connecting to www7.lawrence.edu (www7.lawrence.edu)|143.44.124.14|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www2.lawrence.edu/fast/greggj/CMSC490/shakespeare.zip [following]
--2023-10-15 14:24:52--  https://www2.lawrence.edu/fast/greggj/CMSC490/shakespeare.zip
Resolving www2.lawrence.edu (www2.lawrence.edu)... 143.44.124.14
Connecting to www2.lawrence.edu (www2.lawrence.edu)|

In [2]:
def getWord(file):
    char = file.read(1)
    if not char:
        return ''
    char = char.lower()

    while char < 'a' or char > 'z':
        char = file.read(1)        
        if not char: 
            return ''
        char = char.lower()

    str = ''
    while char >= 'a' and char <= 'z':
        str = str + char
        char = file.read(1)        
        if not char: 
            return str
        char = char.lower()
    
    if char == '’':
        str = ''
        char = file.read(1)
        if not char: 
            return str
        char = char.lower()
        while char >= 'a' and char <= 'z':
            char = file.read(1)        
            if not char: 
                return str
            char = char.lower()
        return getWord(file)
    return str

In [29]:
import numpy as np

def makeSequence(s):
    seq = []
    for c in s:
        position = 0
        oneHot_encoding = np.zeros(27)
        if c == '':
            position = 26
        else:
            position = ord(c) - ord('a')
        oneHot_encoding[position] = 1
        seq.append(oneHot_encoding)
    
    #pad result if number of characters less than 10
    while len(seq) < 10:
        oneHot_encoding = np.zeros(27)
        oneHot_encoding[26] = 1
        seq.append(oneHot_encoding)
    
    seq = np.array(seq)
    return seq

def makeOneSequence(c):
    position = 0
    oneHot_encoding = np.zeros(27)
    if c == '':
        position = 26
    else:
        position = ord(c) - ord('a')
    oneHot_encoding[position] = 1
    return oneHot_encoding    
        

In [30]:
#generate dataset
file = open('shakespeare.txt', 'r')

input_sequences = []
target_characters = []

for n in range(100000):
    word = getWord(file)
    l = len(word)
    if (l < 4 or l > 9 ):
        continue
    curSubString = word[:3]
    for i in range(3, l):
        # Generate the one-hot encoded sequence for the current substring
        input_seq = makeSequence(curSubString)

        # Append the input sequence to the list
        input_sequences.append(input_seq)

        # Append the target character (next character) to the list
        target_characters.append(makeOneSequence(word[i]))

        # Move the substring one character to the right
        curSubString = curSubString + word[i]
    
file.close()

input_sequences = np.array(input_sequences)
target_characters = np.array(target_characters)


In [31]:
input_sequences.shape

(128319, 10, 27)

In [32]:
target_characters.shape

(128319, 27)

In [33]:
#divide the big dataset
x_train = input_sequences[:80000]
x_validation = input_sequences[80000:100000]
x_test = input_sequences[100000:]
y_train = target_characters[:80000]
y_validation = target_characters[80000:100000]
y_test = target_characters[100000:]


In [34]:
from tensorflow.keras import layers

def build_model():
    model = keras.Sequential([
        layers.LSTM(16,input_shape=(10,27),return_sequences=False),
        layers.Dense(27, activation="softmax")
    ])
    model.compile(optimizer="Adam", loss="categorical_crossentropy", metrics=["accuracy"])
    return model

In [35]:
from tensorflow import keras

model = build_model()

callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath="convnet_from_scratch.mdl",
        save_best_only=True,
        monitor="val_loss")
]
history = model.fit(
    x_train,
    y_train,
    epochs=20,
    validation_data= (x_validation, y_validation),
    callbacks=callbacks)

Epoch 1/20
2500/2500 [==============================] - ETA: 0s - loss: 2.5918 - accuracy: 0.2335INFO:tensorflow:Assets written to: convnet_from_scratch.mdl/assets


INFO:tensorflow:Assets written to: convnet_from_scratch.mdl/assets


2500/2500 [==============================] - 12s 4ms/step - loss: 2.5918 - accuracy: 0.2335 - val_loss: 2.3494 - val_accuracy: 0.2998
Epoch 2/20
2490/2500 [============================>.] - ETA: 0s - loss: 2.2522 - accuracy: 0.3414INFO:tensorflow:Assets written to: convnet_from_scratch.mdl/assets


INFO:tensorflow:Assets written to: convnet_from_scratch.mdl/assets


2500/2500 [==============================] - 11s 5ms/step - loss: 2.2518 - accuracy: 0.3415 - val_loss: 2.1682 - val_accuracy: 0.3666
Epoch 3/20
2487/2500 [============================>.] - ETA: 0s - loss: 2.0848 - accuracy: 0.3986INFO:tensorflow:Assets written to: convnet_from_scratch.mdl/assets


...

2500/2500 [==============================] - 13s 5ms/step - loss: 1.6021 - accuracy: 0.5462 - val_loss: 1.7265 - val_accuracy: 0.5124
Epoch 19/20
2491/2500 [============================>.] - ETA: 0s - loss: 1.5896 - accuracy: 0.5493INFO:tensorflow:Assets written to: convnet_from_scratch.mdl/assets


INFO:tensorflow:Assets written to: convnet_from_scratch.mdl/assets


2500/2500 [==============================] - 10s 4ms/step - loss: 1.5896 - accuracy: 0.5493 - val_loss: 1.7172 - val_accuracy: 0.5170
Epoch 20/20
2500/2500 [==============================] - ETA: 0s - loss: 1.5782 - accuracy: 0.5520INFO:tensorflow:Assets written to: convnet_from_scratch.mdl/assets


INFO:tensorflow:Assets written to: convnet_from_scratch.mdl/assets


2500/2500 [==============================] - 14s 6ms/step - loss: 1.5782 - accuracy: 0.5520 - val_loss: 1.7150 - val_accuracy: 0.5201
